In [ ]:
# Install required packages
!pip install "numpy<2" ultralytics opencv-python matplotlib pyyaml torch torchvision kagglehub --quiet

In [ ]:
import kagglehub

# Download dataset from Kaggle
path = kagglehub.dataset_download("fareselmenshawii/face-detection-dataset")

print("Path to dataset files:", path)

# Define paths
TRAIN_PATH = f"{path}/images/train"
VAL_PATH = f"{path}/images/val"

print(f"✅ Train: {TRAIN_PATH}")
print(f"✅ Val: {VAL_PATH}")

In [ ]:
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
import yaml
import shutil
import datetime

from ultralytics import YOLO
from IPython.display import Image, display

## Create YAML Configuration

In [ ]:
# Create data.yaml
data_yaml = {
    'train': TRAIN_PATH,
    'val': VAL_PATH,
    'nc': 1,
    'names': ['Human Face']
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("✅ Created data.yaml:")
with open('data.yaml', 'r') as f:
    print(f.read())

## Training Function

In [ ]:
def yolo_train(model_name, data_yaml, project_name,
               save_dir="/content/trained_yolo_face",
               n_loops=1, epochs_per_loop=5, workers=8):
    """
    Train YOLO model for face detection.
    """
    print(f"📥 Loading YOLO model: {model_name}")
    model = YOLO(model_name)

    os.makedirs(save_dir, exist_ok=True)

    for i in range(1, n_loops + 1):
        now = datetime.datetime.now()
        timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S")
        run_name = f"{project_name}_run_{timestamp_str}"

        print(f"\n🚀 Training Loop {i}/{n_loops} → {run_name}")
        
        model.train(
            data=data_yaml,
            epochs=epochs_per_loop,
            imgsz=640,
            project=project_name,
            name=run_name,
            workers=workers
        )

        run_dir = os.path.join(project_name, run_name)
        weights_dir = os.path.join(run_dir, "weights")
        best_model_path = os.path.join(weights_dir, "best.pt")

        if not os.path.exists(best_model_path):
            raise FileNotFoundError(f"❌ best.pt not found in {weights_dir}")

        print(f"📌 Best model saved at: {best_model_path}")

        final_model_path = os.path.join(save_dir, "best.pt")
        shutil.copy(best_model_path, final_model_path)
        print(f"💾 Model copied to → {final_model_path}")

        model = YOLO(best_model_path)

    total_epochs = n_loops * epochs_per_loop
    print("\n🎉 Training completed!")
    print(f"➡ Total epochs: {total_epochs}")
    print(f"🏁 Final model saved at: {final_model_path}")
    
    return final_model_path

## Start Training

In [ ]:
# Train the model
trained_model_path = yolo_train(
    model_name="yolov8n.pt",
    data_yaml="data.yaml",
    project_name="face_detection",
    save_dir="/content/trained_yolo_face",
    n_loops=1,
    epochs_per_loop=50,
    workers=8
)

## Utility Functions for Inference

In [ ]:
def crop_face(model, image_path, padding_pixels=25):
    """Crop detected face from image."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Cannot read image: {image_path}")
        return None

    results = model(image)
    detections = results[0].boxes.data.cpu().tolist()

    if not detections:
        print(f"⚠️ No face detected in: {os.path.basename(image_path)}")
        return None

    best_detection = max(detections, key=lambda d: d[4])
    x1, y1, x2, y2, score = map(int, best_detection[:5])
    
    print(f"Image '{os.path.basename(image_path)}': Score {score:.2f}")

    img_h, img_w, _ = image.shape

    final_x1 = int(np.clip(x1 - padding_pixels, 0, img_w))
    final_y1 = int(np.clip(y1 - padding_pixels, 0, img_h))
    final_x2 = int(np.clip(x2 + padding_pixels, 0, img_w))
    final_y2 = int(np.clip(y2 + padding_pixels, 0, img_h))
    
    cropped_face = image[final_y1:final_y2, final_x1:final_x2]
    return cropped_face

## Test Predictions

In [ ]:
# Load trained model
model = YOLO("/content/trained_yolo_face/best.pt")

# Get test images
image_folder_path = VAL_PATH
num_files = 10
first_images = []

with os.scandir(image_folder_path) as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith(('.jpg', '.jpeg', '.png')):
            first_images.append(entry.path)
            if len(first_images) == num_files:
                break

print(f"📊 {len(first_images)} images found")

# Run predictions
output_dir = "/content/predictions"
os.makedirs(output_dir, exist_ok=True)

for img_path in first_images:
    results = model.predict(source=img_path, save=True, project=output_dir, name="results", exist_ok=True)

print("✅ Predictions complete!")